In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

---

## Part 1: Why Update Graphs?

### The Problem

```
Scenario: Training loop with changing data

Without Updates (slow):
━━━━━━━━━━━━━━━━━━━━━━━
for epoch in range(100):
    for batch in batches:
        # Rebuild graph every time! (expensive)
        capture_graph(batch)  # ~100ms
        instantiate()         # ~10ms
        launch()              # ~0.1ms

With Updates (fast):
━━━━━━━━━━━━━━━━━━━━━━━
# Build once
capture_graph(first_batch)
instantiate()

for epoch in range(100):
    for batch in batches:
        update_parameters(batch)  # ~1μs
        launch()                  # ~0.1ms
```

---

## Part 2: Updating Kernel Parameters

### 🔷 CUDA C++ Implementation (Primary)

This example shows how to efficiently update kernel parameters without rebuilding the graph - essential for training loops and batch processing.

In [ ]:
%%writefile graph_update_kernel.cu
// graph_update_kernel.cu - Updating kernel parameters
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void scaleAdd(float* data, float scale, float add, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        data[tid] = data[tid] * scale + add;
    }
}

int main() {
    const int N = 1 << 20;
    float *d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    // Initial parameters
    float scale = 2.0f;
    float add = 1.0f;
    int n = N;
    
    cudaStream_t stream;
    cudaStreamCreate(&stream);
    
    // ============================================
    // Capture Initial Graph
    // ============================================
    cudaGraph_t graph;
    cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);
    
    scaleAdd<<<256, 256, 0, stream>>>(d_data, scale, add, n);
    
    cudaStreamEndCapture(stream, &graph);
    
    cudaGraphExec_t graphExec;
    cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0);
    
    // ============================================
    // Method 1: Update via Node
    // ============================================
    
    // Get the kernel node from graph
    size_t numNodes;
    cudaGraphGetNodes(graph, NULL, &numNodes);
    cudaGraphNode_t* nodes = new cudaGraphNode_t[numNodes];
    cudaGraphGetNodes(graph, nodes, &numNodes);
    
    // Find the kernel node
    cudaGraphNode_t kernelNode = NULL;
    for (size_t i = 0; i < numNodes; i++) {
        cudaGraphNodeType type;
        cudaGraphNodeGetType(nodes[i], &type);
        if (type == cudaGraphNodeTypeKernel) {
            kernelNode = nodes[i];
            break;
        }
    }
    
    // Update parameters
    float newScale = 3.0f;
    float newAdd = 2.0f;
    
    cudaKernelNodeParams params;
    cudaGraphKernelNodeGetParams(kernelNode, &params);
    
    // Update kernel arguments
    void* newArgs[] = { &d_data, &newScale, &newAdd, &n };
    params.kernelParams = newArgs;
    
    // Apply update to executable graph
    cudaGraphExecKernelNodeSetParams(graphExec, kernelNode, &params);
    
    // Launch with new parameters!
    cudaGraphLaunch(graphExec, stream);
    cudaStreamSynchronize(stream);
    
    printf("Updated kernel with scale=%.1f, add=%.1f\n", newScale, newAdd);
    
    // ============================================
    // Method 2: cudaGraphExecUpdate (whole graph)
    // ============================================
    
    // Capture a new graph with different parameters
    cudaGraph_t newGraph;
    float anotherScale = 4.0f;
    
    cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);
    scaleAdd<<<256, 256, 0, stream>>>(d_data, anotherScale, add, n);
    cudaStreamEndCapture(stream, &newGraph);
    
    // Update executable from new graph (must have same topology!)
    cudaGraphNode_t errorNode;
    cudaGraphExecUpdateResult updateResult;
    
    cudaGraphExecUpdate(graphExec, newGraph, &errorNode, &updateResult);
    
    if (updateResult == cudaGraphExecUpdateSuccess) {
        printf("Graph updated successfully!\n");
        cudaGraphLaunch(graphExec, stream);
        cudaStreamSynchronize(stream);
    } else {
        printf("Update failed, need to reinstantiate\n");
    }
    
    // Cleanup
    delete[] nodes;
    cudaGraphExecDestroy(graphExec);
    cudaGraphDestroy(graph);
    cudaGraphDestroy(newGraph);
    cudaStreamDestroy(stream);
    cudaFree(d_data);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o graph_update_kernel graph_update_kernel.cu
!./graph_update_kernel

---

## Part 3: Updating Memcpy Nodes

### 🔷 CUDA C++ Implementation (Primary)

This example demonstrates updating data pointers in memcpy nodes - useful for processing multiple buffers with the same graph.

In [ ]:
%%writefile graph_update_memcpy.cu
// graph_update_memcpy.cu - Updating data pointers
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void process(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) data[tid] *= 2.0f;
}

int main() {
    const int N = 1 << 20;
    const size_t BYTES = N * sizeof(float);
    const int NUM_BUFFERS = 4;
    
    // Multiple input/output buffers
    float* h_inputs[NUM_BUFFERS];
    float* h_outputs[NUM_BUFFERS];
    for (int i = 0; i < NUM_BUFFERS; i++) {
        cudaMallocHost(&h_inputs[i], BYTES);
        cudaMallocHost(&h_outputs[i], BYTES);
        for (int j = 0; j < N; j++) h_inputs[i][j] = (float)(i + 1);
    }
    
    float* d_data;
    cudaMalloc(&d_data, BYTES);
    
    cudaStream_t stream;
    cudaStreamCreate(&stream);
    
    // Capture with first buffer
    cudaGraph_t graph;
    cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);
    
    cudaMemcpyAsync(d_data, h_inputs[0], BYTES, cudaMemcpyHostToDevice, stream);
    process<<<256, 256, 0, stream>>>(d_data, N);
    cudaMemcpyAsync(h_outputs[0], d_data, BYTES, cudaMemcpyDeviceToHost, stream);
    
    cudaStreamEndCapture(stream, &graph);
    
    cudaGraphExec_t graphExec;
    cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0);
    
    // ============================================
    // Get memcpy nodes
    // ============================================
    size_t numNodes;
    cudaGraphGetNodes(graph, NULL, &numNodes);
    cudaGraphNode_t* nodes = new cudaGraphNode_t[numNodes];
    cudaGraphGetNodes(graph, nodes, &numNodes);
    
    cudaGraphNode_t h2dNode = NULL, d2hNode = NULL;
    for (size_t i = 0; i < numNodes; i++) {
        cudaGraphNodeType type;
        cudaGraphNodeGetType(nodes[i], &type);
        if (type == cudaGraphNodeTypeMemcpy) {
            // Determine if H2D or D2H by checking parameters
            cudaMemcpy3DParms params;
            cudaGraphMemcpyNodeGetParams(nodes[i], &params);
            if (params.kind == cudaMemcpyHostToDevice) {
                h2dNode = nodes[i];
            } else {
                d2hNode = nodes[i];
            }
        }
    }
    
    // ============================================
    // Process each buffer by updating graph
    // ============================================
    for (int buf = 0; buf < NUM_BUFFERS; buf++) {
        // Update H2D source
        cudaMemcpy3DParms h2dParams = {0};
        h2dParams.srcPtr = make_cudaPitchedPtr(h_inputs[buf], BYTES, N, 1);
        h2dParams.dstPtr = make_cudaPitchedPtr(d_data, BYTES, N, 1);
        h2dParams.extent = make_cudaExtent(BYTES, 1, 1);
        h2dParams.kind = cudaMemcpyHostToDevice;
        cudaGraphExecMemcpyNodeSetParams(graphExec, h2dNode, &h2dParams);
        
        // Update D2H destination
        cudaMemcpy3DParms d2hParams = {0};
        d2hParams.srcPtr = make_cudaPitchedPtr(d_data, BYTES, N, 1);
        d2hParams.dstPtr = make_cudaPitchedPtr(h_outputs[buf], BYTES, N, 1);
        d2hParams.extent = make_cudaExtent(BYTES, 1, 1);
        d2hParams.kind = cudaMemcpyDeviceToHost;
        cudaGraphExecMemcpyNodeSetParams(graphExec, d2hNode, &d2hParams);
        
        // Launch with updated pointers
        cudaGraphLaunch(graphExec, stream);
        cudaStreamSynchronize(stream);
        
        printf("Buffer %d: input=%.0f, output=%.0f\n", 
               buf, h_inputs[buf][0], h_outputs[buf][0]);
    }
    
    // Cleanup
    delete[] nodes;
    cudaGraphExecDestroy(graphExec);
    cudaGraphDestroy(graph);
    cudaStreamDestroy(stream);
    cudaFree(d_data);
    for (int i = 0; i < NUM_BUFFERS; i++) {
        cudaFreeHost(h_inputs[i]);
        cudaFreeHost(h_outputs[i]);
    }
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o graph_update_memcpy graph_update_memcpy.cu
!./graph_update_memcpy

---

## Part 4: Update Limitations

### What Can Be Updated

```
✅ CAN update:
• Kernel arguments (different values)
• Memcpy source/destination pointers
• Memcpy size (same or smaller)
• Memset values

❌ CANNOT update:
• Graph topology (add/remove nodes)
• Kernel function (different kernel)
• Block/grid dimensions
• Dependency structure
• Node types

If you need to change these → Reinstantiate!
```

### Update Error Handling

```cpp
cudaGraphNode_t errorNode;
cudaGraphExecUpdateResult result;

cudaGraphExecUpdate(graphExec, newGraph, &errorNode, &result);

switch (result) {
    case cudaGraphExecUpdateSuccess:
        // Good to go!
        break;
    case cudaGraphExecUpdateError:
        // General error
        break;
    case cudaGraphExecUpdateErrorTopologyChanged:
        // Graph structure changed - must reinstantiate
        cudaGraphExecDestroy(graphExec);
        cudaGraphInstantiate(&graphExec, newGraph, NULL, NULL, 0);
        break;
    case cudaGraphExecUpdateErrorNodeTypeChanged:
        // Node type changed - must reinstantiate
        break;
    case cudaGraphExecUpdateErrorFunctionChanged:
        // Kernel function changed - must reinstantiate
        break;
    case cudaGraphExecUpdateErrorParametersChanged:
        // Parameters incompatible
        break;
}
```

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile graph_updates_exercises.cu
/*
 * CUDA Graph Updates Exercises
 * Exercise 1: Iterative Updates - Measure update overhead
 * Exercise 2: Buffer Rotation - Double-buffering with graph updates
 * Exercise 3: Graceful Fallback - Update with fallback to reinstantiation
 */

#include <cuda_runtime.h>
#include <stdio.h>
#include <chrono>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA error at %s:%d: %s\n", __FILE__, __LINE__, \
                   cudaGetErrorString(err)); \
            exit(1); \
        } \
    } while(0)

// Simple processing kernel
__global__ void processKernel(float* data, int n, float scale) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = data[idx] * scale + 1.0f;
    }
}

// =============================================================
// Exercise 1: Iterative Updates
// Measure the overhead of updating graph parameters 1000 times
// =============================================================
void exercise1_iterative_updates() {
    printf("\n=== Exercise 1: Iterative Updates ===\n");
    
    const int N = 1024 * 1024;
    const int NUM_UPDATES = 1000;
    float *d_data;
    
    CHECK_CUDA(cudaMalloc(&d_data, N * sizeof(float)));
    CHECK_CUDA(cudaMemset(d_data, 0, N * sizeof(float)));
    
    // Create graph via stream capture
    cudaStream_t stream;
    cudaGraph_t graph;
    cudaGraphExec_t graphExec;
    
    CHECK_CUDA(cudaStreamCreate(&stream));
    CHECK_CUDA(cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal));
    
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    processKernel<<<numBlocks, blockSize, 0, stream>>>(d_data, N, 1.0f);
    
    CHECK_CUDA(cudaStreamEndCapture(stream, &graph));
    CHECK_CUDA(cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0));
    
    // Get the kernel node
    size_t numNodes = 0;
    CHECK_CUDA(cudaGraphGetNodes(graph, nullptr, &numNodes));
    printf("Graph has %zu nodes\n", numNodes);
    
    cudaGraphNode_t* nodes = new cudaGraphNode_t[numNodes];
    CHECK_CUDA(cudaGraphGetNodes(graph, nodes, &numNodes));
    
    // Find the kernel node
    cudaGraphNode_t kernelNode = nullptr;
    for (size_t i = 0; i < numNodes; i++) {
        cudaGraphNodeType type;
        CHECK_CUDA(cudaGraphNodeGetType(nodes[i], &type));
        if (type == cudaGraphNodeTypeKernel) {
            kernelNode = nodes[i];
            break;
        }
    }
    
    if (kernelNode == nullptr) {
        printf("No kernel node found!\n");
        return;
    }
    
    // Measure update overhead
    auto start = std::chrono::high_resolution_clock::now();
    
    for (int i = 0; i < NUM_UPDATES; i++) {
        // Get current kernel params and update scale
        cudaKernelNodeParams nodeParams;
        CHECK_CUDA(cudaGraphKernelNodeGetParams(kernelNode, &nodeParams));
        
        float newScale = 1.0f + (float)i * 0.001f;
        void* kernelArgs[] = { &d_data, (void*)&N, &newScale };
        nodeParams.kernelParams = kernelArgs;
        
        CHECK_CUDA(cudaGraphExecKernelNodeSetParams(graphExec, kernelNode, &nodeParams));
    }
    
    auto end = std::chrono::high_resolution_clock::now();
    double updateTime = std::chrono::duration<double, std::micro>(end - start).count();
    
    printf("Total time for %d updates: %.2f µs\n", NUM_UPDATES, updateTime);
    printf("Average time per update: %.2f µs\n", updateTime / NUM_UPDATES);
    
    // Launch once to verify
    CHECK_CUDA(cudaGraphLaunch(graphExec, stream));
    CHECK_CUDA(cudaStreamSynchronize(stream));
    printf("Graph executed successfully after updates\n");
    
    // Cleanup
    delete[] nodes;
    CHECK_CUDA(cudaGraphExecDestroy(graphExec));
    CHECK_CUDA(cudaGraphDestroy(graph));
    CHECK_CUDA(cudaStreamDestroy(stream));
    CHECK_CUDA(cudaFree(d_data));
}

// =============================================================
// Exercise 2: Buffer Rotation
// Implement double-buffering with graph updates
// =============================================================
void exercise2_buffer_rotation() {
    printf("\n=== Exercise 2: Buffer Rotation ===\n");
    
    const int N = 1024 * 1024;
    const int NUM_ITERATIONS = 10;
    float *d_bufferA, *d_bufferB;
    float *d_buffers[2];
    
    CHECK_CUDA(cudaMalloc(&d_bufferA, N * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_bufferB, N * sizeof(float)));
    d_buffers[0] = d_bufferA;
    d_buffers[1] = d_bufferB;
    
    // Initialize buffer A
    float* h_init = new float[N];
    for (int i = 0; i < N; i++) h_init[i] = 1.0f;
    CHECK_CUDA(cudaMemcpy(d_bufferA, h_init, N * sizeof(float), cudaMemcpyHostToDevice));
    
    // Create graph with buffer A
    cudaStream_t stream;
    cudaGraph_t graph;
    cudaGraphExec_t graphExec;
    
    CHECK_CUDA(cudaStreamCreate(&stream));
    CHECK_CUDA(cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal));
    
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    float* currentBuffer = d_bufferA;
    processKernel<<<numBlocks, blockSize, 0, stream>>>(currentBuffer, N, 2.0f);
    
    CHECK_CUDA(cudaStreamEndCapture(stream, &graph));
    CHECK_CUDA(cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0));
    
    // Get kernel node for updates
    size_t numNodes = 0;
    CHECK_CUDA(cudaGraphGetNodes(graph, nullptr, &numNodes));
    cudaGraphNode_t* nodes = new cudaGraphNode_t[numNodes];
    CHECK_CUDA(cudaGraphGetNodes(graph, nodes, &numNodes));
    
    cudaGraphNode_t kernelNode = nullptr;
    for (size_t i = 0; i < numNodes; i++) {
        cudaGraphNodeType type;
        CHECK_CUDA(cudaGraphNodeGetType(nodes[i], &type));
        if (type == cudaGraphNodeTypeKernel) {
            kernelNode = nodes[i];
            break;
        }
    }
    
    // Alternate between buffers
    for (int iter = 0; iter < NUM_ITERATIONS; iter++) {
        int bufferIdx = iter % 2;
        float* activeBuffer = d_buffers[bufferIdx];
        
        // Update kernel params to use current buffer
        cudaKernelNodeParams nodeParams;
        CHECK_CUDA(cudaGraphKernelNodeGetParams(kernelNode, &nodeParams));
        
        float scale = 2.0f;
        int size = N;
        void* kernelArgs[] = { &activeBuffer, &size, &scale };
        nodeParams.kernelParams = kernelArgs;
        
        CHECK_CUDA(cudaGraphExecKernelNodeSetParams(graphExec, kernelNode, &nodeParams));
        CHECK_CUDA(cudaGraphLaunch(graphExec, stream));
        
        printf("Iteration %d: Using buffer %c\n", iter, 'A' + bufferIdx);
    }
    
    CHECK_CUDA(cudaStreamSynchronize(stream));
    printf("Double-buffering completed successfully\n");
    
    // Cleanup
    delete[] h_init;
    delete[] nodes;
    CHECK_CUDA(cudaGraphExecDestroy(graphExec));
    CHECK_CUDA(cudaGraphDestroy(graph));
    CHECK_CUDA(cudaStreamDestroy(stream));
    CHECK_CUDA(cudaFree(d_bufferA));
    CHECK_CUDA(cudaFree(d_bufferB));
}

// =============================================================
// Exercise 3: Graceful Fallback
// Update logic with fallback to reinstantiation when needed
// =============================================================
void exercise3_graceful_fallback() {
    printf("\n=== Exercise 3: Graceful Fallback ===\n");
    
    const int N = 1024;
    float *d_data;
    CHECK_CUDA(cudaMalloc(&d_data, N * sizeof(float)));
    
    cudaStream_t stream;
    cudaGraph_t graph;
    cudaGraphExec_t graphExec;
    
    CHECK_CUDA(cudaStreamCreate(&stream));
    
    // Create initial graph
    CHECK_CUDA(cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal));
    processKernel<<<4, 256, 0, stream>>>(d_data, N, 1.5f);
    CHECK_CUDA(cudaStreamEndCapture(stream, &graph));
    CHECK_CUDA(cudaGraphInstantiate(&graphExec, graph, NULL, NULL, 0));
    
    printf("Initial graph created and instantiated\n");
    
    // Simulate multiple updates with potential failures
    int successfulUpdates = 0;
    int fallbackCount = 0;
    
    for (int i = 0; i < 5; i++) {
        // Create a "new" graph (simulating topology change on some iterations)
        cudaGraph_t newGraph;
        CHECK_CUDA(cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal));
        
        if (i == 2) {
            // Simulate topology change - add extra kernel
            processKernel<<<4, 256, 0, stream>>>(d_data, N, 1.0f);
            processKernel<<<4, 256, 0, stream>>>(d_data, N, 2.0f);
            printf("Iteration %d: Changed topology (2 kernels)\n", i);
        } else {
            processKernel<<<4, 256, 0, stream>>>(d_data, N, 1.0f + i * 0.1f);
            printf("Iteration %d: Same topology\n", i);
        }
        
        CHECK_CUDA(cudaStreamEndCapture(stream, &newGraph));
        
        // Try to update existing graphExec
        cudaGraphExecUpdateResult updateResult;
        cudaGraphNode_t errorNode;
        
        cudaError_t updateErr = cudaGraphExecUpdate(graphExec, newGraph, &errorNode, &updateResult);
        
        if (updateErr == cudaSuccess && updateResult == cudaGraphExecUpdateSuccess) {
            printf("  -> Update succeeded\n");
            successfulUpdates++;
        } else {
            printf("  -> Update failed (result=%d), falling back to reinstantiation\n", updateResult);
            
            // Fallback: destroy old exec and reinstantiate
            CHECK_CUDA(cudaGraphExecDestroy(graphExec));
            CHECK_CUDA(cudaGraphInstantiate(&graphExec, newGraph, NULL, NULL, 0));
            fallbackCount++;
        }
        
        // Launch the graph
        CHECK_CUDA(cudaGraphLaunch(graphExec, stream));
        CHECK_CUDA(cudaStreamSynchronize(stream));
        
        CHECK_CUDA(cudaGraphDestroy(newGraph));
    }
    
    printf("\nSummary:\n");
    printf("  Successful updates: %d\n", successfulUpdates);
    printf("  Fallback reinstantiations: %d\n", fallbackCount);
    
    // Cleanup
    CHECK_CUDA(cudaGraphExecDestroy(graphExec));
    CHECK_CUDA(cudaGraphDestroy(graph));
    CHECK_CUDA(cudaStreamDestroy(stream));
    CHECK_CUDA(cudaFree(d_data));
}

int main() {
    printf("CUDA Graph Updates Exercises\n");
    printf("============================\n");
    
    // Check CUDA device
    int device;
    cudaDeviceProp prop;
    CHECK_CUDA(cudaGetDevice(&device));
    CHECK_CUDA(cudaGetDeviceProperties(&prop, device));
    printf("Device: %s\n", prop.name);
    
    exercise1_iterative_updates();
    exercise2_buffer_rotation();
    exercise3_graceful_fallback();
    
    printf("\n✓ All exercises completed!\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o graph_updates_exercises graph_updates_exercises.cu && ./graph_updates_exercises

### 🔶 Python/Numba Exercises (Optional)

### Exercise 1: Iterative Updates
Create a graph and update kernel parameters 1000 times, measuring update overhead.

### Exercise 2: Buffer Rotation
Implement double-buffering with graph updates.

### Exercise 3: Graceful Fallback
Implement update logic with fallback to reinstantiation when needed.

---

## Key Takeaways

```
┌─────────────────────────────────────────────────────────┐
│                 GRAPH UPDATES                           │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  Per-Node Updates:                                      │
│  • cudaGraphExecKernelNodeSetParams()                   │
│  • cudaGraphExecMemcpyNodeSetParams()                   │
│  • cudaGraphExecMemsetNodeSetParams()                   │
│                                                         │
│  Whole-Graph Update:                                    │
│  • cudaGraphExecUpdate(exec, newGraph, ...)             │
│  • Check result for success/failure                     │
│                                                         │
│  Limitations:                                           │
│  • Topology must stay the same                          │
│  • Kernel function must stay the same                   │
│  • Grid/block dims must stay the same                   │
│                                                         │
│  Performance: Updates are ~1000x faster than rebuild    │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

## Next: Day 4 - Graph Optimization